In [ ]:
#@title <font color='Blue'>**Overheads**</font>

# Author: Lara Hofmann 
# Note: based on code by Yu-Man Tam
# Last updated: 27/07/2022

# References 
# 1. https://github.com/YuMan-Tam/deep-hedging/blob/master/colab/deep_hedging_colab.ipynb
# 2. Deep Hedging (2019, Quantitative Finance) by Buehler et al.
#    https://www.tandfonline.com/doi/abs/10.1080/14697688.2019.1571683

!apt-get update --fix-missing
!pip -q install QuantLib tqdm

# Import custom modules from github
!rm -rf /content/deep_hedging
!git clone -b main https://github.com/lara2302/deep_hedging


import sys, os
sys.path.insert(0, os.getcwd() + "/deep_hedging")

from IPython.display import clear_output

from tqdm import trange

import numpy as np
import pandas as pd
import QuantLib as ql
import tensorflow as tf
import seaborn as sns
from scipy.stats import norm
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, \
                                            ReduceLROnPlateau
from tensorflow.compat.v1.keras.optimizers import Adam
from tensorflow.keras.models import Model

import matplotlib.pyplot as plt

from stochastic_processes import BlackScholesProcess, MertonJumpProcess
from instruments import EuropeanCall, MertonJumpCall, MertonJumpPut, payoff_barrier
from deep_hedging import Deep_Hedging_Model, Delta_SubModel, imp_sampling
from loss_metrics import Entropy, ExpectedShortfall, variance_optimal, \
                         variance_optimal_with_ES
from utilities import train_test_split
from traditional_hedging import Merton_process, Option_param, Merton_pricer

%load_ext autoreload

clear_output()
print("\nFinish installing and importing all necessary libraries!")

In [ ]:
#@title <font color='Blue'>**Input: General**</font>

## GENERAL INPUT VARIABLES

# Jump parameters (Merton Model)
lam = 5                 # Jump intensity
muJ = -0.05             # Jump mean
sigJ = 0.1              # Jump volatility

# Underlying and Option parameters
sig = 0.1               # Implied volatility
S0 = 1.0                # Value at t=0 of the underlying
# For simplicity: set S0=1

K = S0*1.0              # Strike
maturity = 60/365       # Maturity of the option
risk_free = 0.0         # Risk-free rate
dividend = 0.0          # Continuous dividend yield
B = S0*1.2              # Value of the barrier
optiontype = "European" # Type of option: Barrier or European
putorcall = "call"      # Type of option: put or call

if optiontype == "Barrier":
    is_barrier = True
elif optiontype == "European":
    is_barrier = False

if putorcall == "call":
    payoff_func = lambda x: -np.maximum(x - K, 0.0) # Call payoff
elif putorcall == "put":
    payoff_func = lambda x: -np.maximum(K - x, 0.0) # Put payoff

# Hedging parameters
K_hedge = S0*0.9        # Strike of option used for hedge
time_step = 60          # Hedging frequency
dt = maturity/time_step

# Plot
x_limits = [-0.1,0.1]   # X limits for plots

In [ ]:
#@title <font color='Blue'>**Input: Deep-Hedging**</font>

## INPUT VARIABLES: DEEP-HEDGING
Ktrain = 1*(10**5)      # Size of training sample.
Ktest_ratio = 0.2       # Fraction of training sample as testing sample.

# Total obs = Training + Testing
nobs = int(Ktrain*(1+Ktest_ratio)) 

# Proportional transaction cost.
epsilon = np.array([[0.0,0.0]]) # for two hedging instruments (underlying and option)
epsilon_simple = np.array([0.0]) # for one hedging instrument (underlying)

# Information set (in string)
# Choose from: S, log_S, normalized_log_S (by S0)
information_set = "normalized_log_S"

# Loss function
alpha = 0.95            # Loss parameter for Expected Shortfall
loss_lam = 0.1          # Lagrange parameter for Variance Optimal with ES

# Neural network (NN) structure
m = 15                  # Number of neurons in each hidden layer.
d = 3                   # Number of hidden layers (Excl. input & output layer)         

# Neural network training parameters
lr = 1e-2               # Learning rate
batch_size = 1024       # Batch size
epochs = 50             # Number of epochs

# Other parameters
use_batch_norm = False
kernel_initializer = "he_uniform"

#activation_dense = "leaky_relu" 
activation_dense = "tanh"
activation_output = "sigmoid"
final_period_cost = False

delta_constraint = (0.0, 1.0)
delta_constraint_option = (-1.0,1.0)
share_strategy_across_time = False
cost_structure = "proportional"

In [ ]:
#@title <font color='Blue'>**Input: Finite-Differences-Hedging**</font>

## INPUT VARIABLES: FINITE-DIFFERENCES-HEDGING
# Discretization parameters
Nspace = 1200           # Space discretization
Ntime = 1000            # Time discretization
t_vec, dt_FD = np.linspace(0, maturity, Ntime, 
                           retstep=True) # Vector of time discretization

# Simulation parameters
num_paths = int(Ktest_ratio*Ktrain)

## Data Generation

In [ ]:
#@title <font color='Blue'>**Generate Random Paths of Jump Stock Prices.**</font>
np.random.seed(123)
S_result = MertonJumpProcess(S0=S0,mu=1.5,r=risk_free,sig=sig,lam=lam,\
                      muJ=muJ,sigJ=sigJ,risk_neutral=True).gen_path(length=maturity,time_step=time_step,num_paths=nobs)
S_left = S_result[1]
S = S_result[0]
Number_jumps = S_result[2]
Z_rv = S_result[3]

# Plot
np.random.seed(123)
nobs_plot = 20
S_plot = MertonJumpProcess(S0=S0,mu=0.0,r=risk_free,sig=sig,lam=lam,\
                      muJ=muJ,sigJ=sigJ,risk_neutral=True).gen_path(length=maturity,time_step=time_step,num_paths=nobs_plot)
S_plot = S_plot[0]

clear_output()

print("S0 = " + str(S0))
print("sigma = " + str(sig))
print("risk_free = " + str(risk_free))
print("Jump intensity = " + str(lam))
print("Jump volatility = " + str(sigJ))
print("Jump mean = " + str(muJ))
print("Number of time steps = " + str(time_step))
print("Length of each time step = " + "1/365")
print("Simulation Done!")

fig, ax = plt.subplots(figsize=(9,5), facecolor='w')
plt.plot(np.transpose(S_plot))
plt.title("Merton Jump Process Simulation",fontweight='bold')
ax.set_xlabel('Time [days]')
ax.set_ylabel('Price')
plt.show()

In [ ]:
#@title <font color='Blue'>**More Plots**</font>
fig, ax = plt.subplots(figsize=(9,5), facecolor='w')
plt.plot(np.transpose(S[1:200,:]))
plt.title("Merton Jump Process Simulation",fontweight='bold')
ax.set_xlabel('Time in days')
ax.set_ylabel('Price')
plt.show()

In [ ]:
#@title <font color='Blue'>**Importance Sampling**</font>
importance_sampling = False
impsamp_type = "normal" # "normal" or "poisson" or "both"
mu_new = 1.5
lam_new = 10

if importance_sampling:
    likelihood = imp_sampling(impsamp_type=impsamp_type,Z=Z_rv,mu=mu_new,
                              N_jumps=Number_jumps,lam_new=lam_new,
                              sig=sig,risk_free=risk_free,lam=lam,muJ=muJ,
                              sigJ=sigJ,time_step=time_step,maturity=maturity,
                              Ktrain=Ktrain)
    print(likelihood)

In [ ]:
#@title <font color='Blue'>**Compute Merton-Call-Price for S_jump.**</font>
# Different option as additional hedging instrument (different strike)
if putorcall == "call":
    Hedge_call = MertonJumpCall(S=S,
                              strike=K_hedge,
                              T=maturity,
                              risk_free=risk_free,
                              sigma=sig,
                              jump_mean=muJ,
                              jump_volatility=sigJ,
                              jump_intensity=lam)
    C_hedge = Hedge_call.MertonCall(cutoff=3)

    # Get price of option (hedging instrument) and option to be hedged
    C0_hedge = C_hedge[0,0]               # hedging instrument

    # Position to be hedged
    price_C = MertonJumpCall(S=S[0:1,:],
                            strike=K,
                            T=maturity,
                            risk_free=risk_free,
                            sigma=sig,
                            jump_mean=muJ,
                            jump_volatility=sigJ,
                            jump_intensity=lam)
    price_C = price_C.MertonCall()
    price_C0 = price_C[0,0]
elif putorcall == "put":
    Hedge_put = MertonJumpPut(S=S,
                              strike=K_hedge,
                              T=maturity,
                              risk_free=risk_free,
                              sigma=sig,
                              jump_mean=muJ,
                              jump_volatility=sigJ,
                              jump_intensity=lam)
    C_hedge = Hedge_put.MertonPut(cutoff=3)

    # Get price of option (hedging instrument) and option to be hedged
    C0_hedge = C_hedge[0,0]               # hedging instrument

    # Position to be hedged
    price_C = MertonJumpPut(S=S[0:1,:],
                            strike=K,
                            T=maturity,
                            risk_free=risk_free,
                            sigma=sig,
                            jump_mean=muJ,
                            jump_volatility=sigJ,
                            jump_intensity=lam)
    price_C = price_C.MertonPut()
    price_C0 = price_C[0,0]

clear_output()

print("The price of the " + putorcall + " option at t=0 is " + 
      str(np.round(price_C0,decimals=2)) + ".")
print("The percentage of positive payoffs is " + 
      str(np.round(np.mean(-payoff_func(S[:,-1])>0),decimals=2)) + "%.")

In [ ]:
price_C = MertonJumpCall(S=S[0:1,:],
                         strike=K,
                         T=maturity,
                         risk_free=risk_free,
                         sigma=sig,
                         jump_mean=muJ,
                         jump_volatility=sigJ,
                         jump_intensity=lam)
price_C = price_C.MertonCall()
price_C0 = price_C[0,0]

In [ ]:
## For barrier option
if is_barrier:
    price_C0 = np.mean(-payoff_barrier(S,K,B,putorcall))
    print("The naive MC estimate of the Barrier price is " + str(np.round(price_C0,3)) + ".")

## Merton-Jump-Model using one hedging instrument (Stock)

In [ ]:
#@title <font color='Blue'>**Prepare jump data to be fed into the deep hedging algorithm.**</font>

if optiontype == "European":
    payoff_T_jump = payoff_func(S[:,-1])    # Payoff of the call option
elif optiontype == "Barrier":
    payoff_T_jump = payoff_barrier(S,K,B,putorcall)   # Payoff of the Barrier option

trade_set_jump =  np.stack((S),axis=1)      # Trading set

if information_set is "S":
  I_jump =  np.stack((S),axis=1)            # Information set
elif information_set is "log_S":
  I_jump =  np.stack((np.log(S)),axis=1)
elif information_set is "normalized_log_S":
  I_jump =  np.stack((np.log(S/S0)),axis=1)


# In case of a barrier option
if is_barrier:
    I_barrier_hit = np.zeros(np.shape(trade_set_jump))
    npaths = np.shape(trade_set_jump)[1]

    for j in range(npaths):
      for i in range(time_step+1):
          if trade_set_jump[i,j]>= B:
              I_barrier_hit[i:,j] = 1
              break

    I_barrier_hit = I_barrier_hit.astype(float)  
    # In case of standardization
    # I_barrier_hit = (I_barrier_hit - np.mean(I_barrier_hit))/np.std(I_barrier_hit)


# Structure of xtrain:
#   1) Trade set: [S]
#   2) Information set: [I]
#   2.1) If Barrier: Barrier_hit [B] 
#   3) payoff (dim = 1)

N = time_step
x_all_jump = []
for i in range(N+1):
  x_all_jump += [trade_set_jump[i,:,None]]
  if i != N:
    x_all_jump += [I_jump[i,:,None]]
    if is_barrier:
        x_all_jump += [I_barrier_hit[i,:,None]]
x_all_jump += [payoff_T_jump[:,None]]


# Split the entire sample into a training sample and a testing sample.
test_size = num_paths                       # Number of samples for experiments
[xtrain1, xtest1] = train_test_split(x_all_jump, test_size=test_size)
[S_train_jump, S_test_jump] = train_test_split([S], test_size=test_size)
[option_train_jump, option_test_jump] = train_test_split([x_all_jump[-1]], 
                                                         test_size=test_size)
[S_train_left, S_test_left] = train_test_split([S_left], test_size=test_size)
S_test = S_test_jump[0]                     # Test data set: S_t
S_test_left = S_test_left[0]                # Test data set: left limit S_t-

print("Finish preparing data!")

In [ ]:
#@title <font color='Blue'>**Plot payoff.**</font>

fig_PnL = plt.figure(dpi= 125, facecolor='w')
fig_PnL.suptitle("Payoff Distribution (for shorted position)\n", \
        fontweight="bold")
ax = fig_PnL.add_subplot()
ax.set_xlabel("Payoff")
ax.set_ylabel("Frequency")
ax.hist(payoff_T_jump, bins=30, \
            label=["Payoff Distribution"])
ax.legend()
plt.show()

In [ ]:
#@title <font color='Blue'>**Run the Deep Hedging Algorithm (Simple Network) on jump data!**</font>
%autoreload 2

optimizer = Adam(learning_rate=lr)
num_instr = 1


## Setup the model
model_simple_jump = Deep_Hedging_Model(N=N, d=d, m=m, num_instr=num_instr, 
                                       is_barrier = is_barrier,
                                       risk_free=risk_free, dt = dt, 
                                       strategy_type="simple", 
                                       epsilon = epsilon_simple,
                                       use_batch_norm = use_batch_norm, 
                                       kernel_initializer = kernel_initializer,
                                       activation_dense = activation_dense, 
                                       activation_output = activation_output,
                                       final_period_cost = final_period_cost, 
                                       delta_constraint = delta_constraint,
                                       share_strategy_across_time = 
                                       share_strategy_across_time,
                                       cost_structure = cost_structure)


## Set up loss function
loss_jump = variance_optimal(model_simple_jump.output, 
                             p0=-np.mean(option_train_jump))

model_simple_jump.add_loss(loss_jump)


## Compile model
model_simple_jump.compile(optimizer=optimizer)


## Fit the model
if importance_sampling:
    history_model1 = model_simple_jump.fit(x=xtrain1, batch_size=batch_size, epochs=epochs,
                                           validation_split=0.1, verbose=1,sample_weight=likelihood)
else: 
    history_model1 = model_simple_jump.fit(x=xtrain1, batch_size=batch_size, epochs=epochs,
                                           validation_split=0.1, verbose=1)
clear_output()

print("Finished running deep hedging algorithm! (Simple Network)")

Finished running deep hedging algorithm! (Simple Network)


In [ ]:
#@title <font color='Blue'>**Plot Loss Function**</font>
fig, ax = plt.subplots(figsize=(9,5), facecolor='w')
plt.plot(history_model1.epoch, history_model1.history['loss'], label = "Training")
plt.plot(history_model1.epoch, history_model1.history['val_loss'], label = "Validation")
plt.title("Training and Validation Loss: Hedging with Underlying",fontweight='bold')
plt.legend()
ax.set_xlabel('Epochs')
ax.set_ylabel('Loss')
plt.show()

In [ ]:
#@title <font color='Green'>**Reccurent Network?**</font>
run_recurrent = False

In [ ]:
#@title <font color='Blue'>**Run the Deep Hedging Algorithm (Recurrent Network) on jump data!**</font>
%autoreload 2

if run_recurrent == True: 
    optimizer = Adam(learning_rate=lr)
    num_instr = 1

    ## Setup the model
    model_recurrent_jump = Deep_Hedging_Model(N=N, d=d, m=m, num_instr=num_instr,
                                              is_barrier=is_barrier, 
                                              risk_free=risk_free, dt = dt, 
                                              strategy_type="recurrent", 
                                              epsilon = epsilon_simple, 
                                              use_batch_norm = use_batch_norm, 
                                              kernel_initializer = kernel_initializer, 
                                              activation_dense = activation_dense, 
                                              activation_output = activation_output,
                                              final_period_cost = final_period_cost, 
                                              delta_constraint = delta_constraint, 
                                              share_strategy_across_time = 
                                              share_strategy_across_time, 
                                              cost_structure = cost_structure)


    ## Set up loss function
    loss_jump = variance_optimal(model_recurrent_jump.output, 
                             p0=-np.mean(option_train_jump))


    ## Compile the model
    model_recurrent_jump.add_loss(loss_jump)
    model_recurrent_jump.compile(optimizer=optimizer)


    ## Fit the model
    model_recurrent_jump.fit(x=xtrain1, batch_size=batch_size, epochs=epochs,
                             validation_split=0.1, verbose=1)

    clear_output()

    print("Finished running deep hedging algorithm! (Recurrent Network)")
else: print("Recurrent deep hedging algorithm is set not to be run.")

In [ ]:
#@title <font color='Blue'>**Plot Results**</font>
if run_recurrent:
  PNL_model1_simple = model_simple_jump(xtest1).numpy().squeeze() + price_C0
  PNL_model1_recurrent = model_recurrent_jump(xtest1).numpy().squeeze() + price_C0

  # Plot Simple Network PnL vs Recurrent Network PnL (with BS_price charged on both).
  fig_nn = plt.figure(dpi= 125, facecolor='w')
  fig_nn.suptitle("Simple Network PnL vs Recurrent Network PnL \n ", \
        fontweight="bold")
  ax = fig_nn.add_subplot()
  ax.set_title("epsilon = " + str(epsilon), fontsize=8)
  ax.set_xlabel("PnL")
  ax.set_ylabel("Frequency")
  ax.hist((PNL_model1_simple,PNL_model1_recurrent), bins=30, \
        label=["Simple Network PnL", "Recurrent Network PnL"])
  ax.legend()
  plt.show()
else:
  PNL_model1_simple = model_simple_jump(xtest1).numpy().squeeze() + price_C0
  fig_PnL = plt.figure(dpi= 125, facecolor='w')
  fig_PnL.suptitle("Simple Network PnL \n", \
        fontweight="bold")
  ax = fig_PnL.add_subplot()
  ax.set_xlabel("PnL")
  ax.set_ylabel("Frequency")
  ax.hist(PNL_model1_simple, bins=30, \
            label=["Simple Network PnL"])
  ax.legend()
  plt.show()

In [ ]:
df_describe = pd.DataFrame(PNL_model1_simple)
df_describe.describe()

In [ ]:
#@title <font color='Blue'>**Plot Hedging Strategy I**</font>
for days_from_today in (29,int(time_step/2),time_step-1):
  tau = (N-days_from_today)*dt
    
  min_S = S_test[:,days_from_today].min()
  max_S = S_test[:,days_from_today].max()
  S_range = np.linspace(min_S,max_S,101)

  # Attention: Need to transform it to be consistent with the information set.
  if information_set is "S":
    I_range =  S_range # Information set
  elif information_set is "log_S":
    I_range =  np.log(S_range)
  elif information_set is "normalized_log_S":
    I_range =  np.log(S_range/S0)    

  if is_barrier:    
    barrier_hit = (S_range >= B).astype(float)  
      
  # Compute deep-hedging strategy.
  if share_strategy_across_time is False:
    submodel = Delta_SubModel(model = model_simple_jump, \
                      days_from_today = days_from_today)
  else:
    submodel = Delta_SubModel(model = model_simple_jump, \
                      days_from_today = 0)

  if is_barrier:
    nn_delta = submodel([I_range,barrier_hit])
  else:  
    nn_delta = submodel(I_range)

  # Create a plot of deep-hedging strategy.
  fig_delta = plt.figure(dpi= 125, facecolor='w')
  fig_delta.suptitle("Deep Hedging Strategy \n", \
        fontweight="bold")
  ax_delta = fig_delta.add_subplot()
  ax_delta.set_title("Simple Network Structure with " + \
              "t=" + str(days_from_today) , fontsize=8)
  ax_delta.set_xlabel("Price of the Underlying Asset")
  ax_delta.set_ylabel("Hedging Strategy")
  ax_delta.scatter(S_range,nn_delta, c="red", s=2, label="Deep Hedging Delta")
  ax_delta.legend()
  plt.show()

In [ ]:
#@title <font color='Blue'>**Plot Hedging Strategy II**</font>
from mpl_toolkits import mplot3d
from matplotlib import cm

fig = plt.figure(figsize=(10,7), facecolor='w')
ax = fig.add_subplot(111, projection='3d')
min_S = S_test[:,59].min()
max_S = S_test[:,59].max() 
S_range = np.linspace(min_S,max_S,10001)      
T_plot, S_plot = np.meshgrid(np.linspace(0, maturity, time_step), S_range)

nn_delta = np.zeros((len(S_range),time_step))
for t in range(time_step):
  tau = (N-t)*dt

  # Attention: Need to transform it to be consistent with the information set.
  if information_set is "S":
    I_range =  S_range # Information set
  elif information_set is "log_S":
    I_range =  np.log(S_range)
  elif information_set is "normalized_log_S":
    I_range =  np.log(S_range/S0)  

  if is_barrier: 
    barrier_hit = (S_range >= B).astype(float) 

  # Compute deep-hedging strategy.    
  if share_strategy_across_time is False:
    submodel = Delta_SubModel(model = model_simple_jump, \
                      days_from_today = t)
  else:
    submodel = Delta_SubModel(model = model_simple_jump, \
                      days_from_today = 0)

  if is_barrier:  
    nn_delta_t = submodel([I_range,barrier_hit])
  else:
    nn_delta_t = submodel(I_range)

  nn_delta[:,t] = nn_delta_t.numpy().reshape(len(S_range),)

# 3-dimensional plot
ax.plot_surface(S_plot[:,4:], T_plot[:,4:], nn_delta[:,4:], cmap=cm.ocean, rcount = 100, ccount = 100,antialiased=True)
ax.set_title("Deep Hedging: Strategy",fontweight="bold",y=1.08, fontsize=12)
ax.zaxis.set_rotate_label(False) 
ax.set_xlabel("S"); ax.set_ylabel("t"); ax.set_zlabel("Hedge",rotation=90)
ax.view_init(30, 70)                        # rotates the 3d plot
plt.show() 

##Merton-Jump-Model using two hedging instruments (stock and option)

In [ ]:
#@title <font color='Blue'>**Prepare jump data to be fed into the deep hedging algorithm.**</font>

if optiontype == "European":
    payoff_T = payoff_func(S[:,-1])           # Payoff of the call option
elif optiontype == "Barrier":
    payoff_T = payoff_barrier(S,K,B,putorcall)           # Payoff of the Barrier option


trade_set =  np.stack((S),axis=1)             # Trading set
trade_set2 = np.stack((C_hedge),axis=1)

if information_set is "S":
  I =  np.stack((S),axis=1)                   # Information set for stock
elif information_set is "log_S":
  I =  np.stack((np.log(S)),axis=1)
elif information_set is "normalized_log_S":
  I =  np.stack((np.log(S/S0)),axis=1)

if information_set is "S":
  I2 =  np.stack((C_hedge),axis=1)            # Information set for option
elif information_set is "log_S":
  I2 =  np.stack((C_hedge),axis=1)
elif information_set is "normalized_log_S":
  I2 =  np.stack((C_hedge/C0_hedge),axis=1)


# In case of a barrier option
if is_barrier:
    I_barrier_hit = np.zeros(np.shape(trade_set_jump))
    npaths = np.shape(trade_set_jump)[1]
    for j in range(npaths):
      for i in range(time_step+1):
          if trade_set_jump[i,j]>= B:
              I_barrier_hit[i:,j] = 1
              break

    I_barrier_hit = I_barrier_hit.astype(float)  
    # In case of standardization
    # I_barrier_hit = (I_barrier_hit - np.mean(I_barrier_hit))/np.std(I_barrier_hit)


# Structure of xtrain:
#   1) Trade set: [S,C]
#   2) Information set: [I]
#   2.1) If Barrier: Barrier_hit [B] 
#   3) payoff (dim = 1)

# Multiple inputs for training have to be given as a list.
N = time_step
x_all = []
for i in range(N+1):
  x_all += [np.concatenate([trade_set[i,:,None],trade_set2[i,:,None]],axis=1)]
  if i != N:
    x_all += [np.concatenate([I[i,:,None],I2[i,:,None]],axis=1)]
    if is_barrier:
        x_all += [I_barrier_hit[i,:,None]]
x_all += [payoff_T[:,None]]

# Split the entire sample into a training sample and a testing sample.
test_size = num_paths
[xtrain2, xtest2] = train_test_split(x_all, test_size=test_size)
[Sjump_train, Sjump_test] = train_test_split([S], test_size=test_size)
[option_train, option_test] = train_test_split([x_all[-1]], test_size=test_size)
S_test = Sjump_test[0]                      # Test data set: S_t
S_test_left = Sjump_test[0]                 # Test data set: left limit S_t-


print("Finish preparing data!")

In [ ]:
#@title <font color='Blue'>**Run the Deep Hedging Algorithm (Simple Network)!**</font>
%autoreload 2

optimizer = Adam(learning_rate=lr)
num_instr = 2


## Setup the model
model_simple_jump_2 = Deep_Hedging_Model(N=N, d=d, m=m, num_instr=num_instr, 
                                        is_barrier = is_barrier,
                                        risk_free=risk_free, dt = dt, 
                                        strategy_type="simple", epsilon = epsilon,
                                        use_batch_norm = use_batch_norm, 
                                        kernel_initializer = kernel_initializer,
                                        activation_dense = activation_dense, 
                                        activation_output = activation_output,
                                        final_period_cost = final_period_cost, 
                                        delta_constraint = delta_constraint_option,
                                        share_strategy_across_time = 
                                        share_strategy_across_time,
                                        cost_structure = cost_structure)


## Set up loss function
loss = variance_optimal(model_simple_jump_2.output, 
                        p0=-np.mean(option_train))

model_simple_jump_2.add_loss(loss)


## Compile model
model_simple_jump_2.compile(optimizer=optimizer)


## Fit the model
if importance_sampling:
    history_model_2 = model_simple_jump_2.fit(x=xtrain2, batch_size=batch_size, epochs=epochs,
                                           validation_split=0.1, verbose=1,sample_weight=likelihood)
else: 
    history_model_2 = model_simple_jump_2.fit(x=xtrain2, batch_size=batch_size, epochs=epochs,
                                           validation_split=0.1, verbose=1)
clear_output()

print("Finished running deep hedging algorithm! (Simple Network)")

In [ ]:
#@title <font color='Blue'>**Plot Loss Function**</font>
fig, ax = plt.subplots(figsize=(9,5), facecolor='w')
plt.plot(history_model_2.epoch, history_model_2.history['loss'], label = "Training")
plt.plot(history_model_2.epoch, history_model_2.history['val_loss'], label = "Validation")
plt.title("Training and Validation Loss: Hedging with Underlying and Option",fontweight='bold')
plt.legend()
ax.set_xlabel('Epochs')
ax.set_ylabel('Loss')
plt.show()

In [ ]:
#@title <font color='Green'>**Reccurent Network?**</font>
run_recurrent = False

In [ ]:
#@title <font color='Blue'>**Run the Deep Hedging Algorithm (Recurrent Network)!**</font>
if run_recurrent == True:
    optimizer = Adam(learning_rate=lr)
    num_instr = 2

    ## Setup the model
    model_recurrent_jump_2 = Deep_Hedging_Model(N=N, d=d, m=m, num_instr=num_instr, 
                                                is_barrier = is_barrier,
                                                risk_free=risk_free, dt = dt, 
                                                strategy_type="recurrent", 
                                                epsilon = epsilon, 
                                                use_batch_norm = use_batch_norm, 
                                                kernel_initializer = kernel_initializer, 
                                                activation_dense = activation_dense, 
                                                activation_output = activation_output,
                                                final_period_cost = final_period_cost, 
                                                delta_constraint = delta_constraint_option, 
                                                share_strategy_across_time = 
                                                share_strategy_across_time, 
                                                cost_structure = cost_structure)


    ## Set up loss function
    loss = variance_optimal(model_recurrent_jump_2.output, 
                            p0=-np.mean(option_train))
    model_recurrent_jump_2.add_loss(loss)


    ## Compile the model
    model_recurrent_jump_2.compile(optimizer=optimizer)


    ## Fit the model
    history = model_recurrent_jump_2.fit(x=xtrain2, batch_size=batch_size, epochs=50, \
              validation_split=0.1, verbose=1)

    clear_output()
    print("Finished running deep hedging algorithm! (Recurrent Network)")

    # Loss plot.
    plt.plot(history.epoch, history.history['loss'], label = "Training")
    plt.plot(history.epoch, history.history['val_loss'], label = "Validation")
    plt.title("Training and Validation Loss")
    plt.legend()
    plt.show()
else: print("Recurrent deep hedging algorithm is set not to be run.")

In [ ]:
#@title <font color='Blue'>**Plot Results**</font>
if run_recurrent == True:
  PNL_model2_simple = model_simple_jump_2(xtest2).numpy().squeeze() + price_C0
  PNL_model2_recurrent = model_recurrent_jump_2(xtest2).numpy().squeeze() + price_C0

  # Plot Simple Network PnL vs Recurrent Network PnL (with BS_price charged on both).
  fig_nn = plt.figure(dpi= 125, facecolor='w')
  fig_nn.suptitle("Simple Network PnL vs Recurrent Network PnL \n ", \
        fontweight="bold")
  ax = fig_nn.add_subplot()
  ax.set_title("epsilon = " + str(epsilon), fontsize=8)
  ax.set_xlabel("PnL")
  ax.set_ylabel("Frequency")
  ax.hist((PNL_model2_simple,PNL_model2_recurrent), bins=30, \
        label=["Simple Network PnL", "Recurrent Network PnL"],log=False)
  ax.legend()
  plt.show()
else:
  PNL_model2_simple = model_simple_jump_2(xtest2).numpy().squeeze() - np.mean(option_train)#price_C0
  fig_PnL = plt.figure(dpi= 125, facecolor='w')
  fig_PnL.suptitle("Simple Network PnL \n", \
        fontweight="bold")
  ax = fig_PnL.add_subplot()
  ax.set_xlabel("PnL")
  ax.set_ylabel("Frequency")
  ax.hist(PNL_model2_simple, bins=30, \
            label=["Simple Network PnL"])
  ax.legend()
  plt.show()

In [ ]:
df_describe = pd.DataFrame(PNL_model2_simple)
df_describe.describe()

In [ ]:
#@title <font color='Blue'>**Plot Hedging Strategy II**</font>
#fig = plt.figure(figsize=(10,10))
from mpl_toolkits import mplot3d
from matplotlib import cm

fig = plt.figure(figsize=(10,7), facecolor='w')
ax = fig.add_subplot(111, projection='3d')
min_S = S_test[:,59].min()
max_S = S_test[:,59].max() 
S_range = np.linspace(min_S,max_S,101) 
S_set = S_range.reshape(len(S_range),1)*np.ones((len(S_range),61))   
Hedge_call = MertonJumpCall(S=S_set,
                              strike=K_hedge,
                              T=maturity,
                              risk_free=risk_free,
                              sigma=sig,
                              jump_mean=muJ,
                              jump_volatility=sigJ,
                              jump_intensity=lam)
C_set = Hedge_call.MertonCall(cutoff=3)
clear_output()
T_plot, S_plot = np.meshgrid(np.linspace(0, maturity, time_step), S_range)

nn_delta = np.zeros((len(S_range),time_step))

for t in range(time_step):
  tau = (N-t)*dt
  C_range = C_set[:,t]

  # Attention: Need to transform it to be consistent with the information set.
  if information_set is "S":
    I_range =  S_range # Information set
  elif information_set is "log_S":
    I_range =  np.log(S_range)
  elif information_set is "normalized_log_S":
    I_range =  np.log(S_range/S0)   

  if information_set is "S":
    I2 =  C_range           # Information set for option
  elif information_set is "log_S":
    I2 =  C_range
  elif information_set is "normalized_log_S":
    I2 =  C_range/C0_hedge       

  if is_barrier:
    barrier_hit = (S_range >= B).astype(float) 
      
  # Compute deep-hedging strategy.    
  if share_strategy_across_time is False:
    submodel = Delta_SubModel(model = model_simple_jump_2, \
                      days_from_today = t)
  else:
    submodel = Delta_SubModel(model = model_simple_jump_2, \
                      days_from_today = 0)
  I_total = np.stack([I_range,I2],axis=1)

  if is_barrier:
    nn_delta_t = submodel([I_total,barrier_hit])
  else:
    nn_delta_t = submodel(I_total)

  nn_delta_t_option = nn_delta_t[:,0] 
  nn_delta[:,t] = nn_delta_t_option.numpy().reshape(len(S_range),)

# 3-dimensional plot.
ax.plot_surface(S_plot[:,4:], T_plot[:,4:],nn_delta[:,4:], cmap=cm.ocean, rcount = 100, ccount = 100,antialiased=True)
ax.set_title("Deep Hedging: Strategy",fontweight="bold",y=1.08, fontsize=12)
ax.zaxis.set_rotate_label(False) 
ax.set_xlabel("S"); ax.set_ylabel("t"); ax.set_zlabel("Hedge",rotation=90)
ax.view_init(30,70)                        # rotates the 3d plot
plt.show() 

##Delta Hedge and Simple Quadratic Hedge

In [ ]:
#@title <font color='Blue'>**Compute Option Prices via Finite Differences.**</font>
opt_param = Option_param(S0=S0, K=K, T=maturity, exercise="European", 
                         payoff=putorcall)
Merton_param = Merton_process(r=risk_free, sig=sig, lam=lam, muJ=muJ, 
                              sigJ=sigJ)
Merton = Merton_pricer(opt_param, Merton_param,optiontype=optiontype,B=B,forhedge=False)
p = Merton.PIDE_price((Nspace,Ntime), Time=False) # Price at t=0
V = p[1] # Price matrix for all time steps and all prices of S
x = p[2] # Vector of space discretization of log(S)

print("Finished computing the Option Prices via FD!")

In [ ]:
# Compare computed option prices
if optiontype == "Barrier":
    b_corr = B*np.exp(0*0.5826*sig*dt_FD)
    price_C0_b = np.mean(-payoff_barrier(S,K,b_corr,putorcall))
    print(price_C0_b)
print(p[0])

In [ ]:
#@title <font color='Blue'>**Run Delta Hedging!**</font>
PNL_delta_multi = np.zeros(num_paths)

for i in trange(num_paths):
    S_left_i = S_test_left[i,:].reshape(1,time_step+1)
    S_i = S_test[i,:].reshape(1,time_step+1)
    delta = Merton.delta_hedge(S_left_i)
    cost = 0.0

    for j in range(time_step):
        PNL_delta_multi[i] += delta[j]*(S_i[0,j+1]-S_i[0,j])
        if j == 0:
          cost += np.abs(delta[j])*epsilon[0][0]*S_i[0,j]
        else: 
          cost += np.abs(delta[j]-delta[j-1])*epsilon[0][0]*S_i[0,j]

    if optiontype == "European":
        payoff = -payoff_func(S_i[0,-1])
    elif optiontype == "Barrier":
        payoff = -payoff_barrier(S_i,K,B,putorcall)
    PNL_delta_multi[i] = PNL_delta_multi[i] - payoff + p[0] - cost

clear_output()
print("Finished computing the Delta-Hedging-Strategy!")

In [ ]:
#@title <font color='Blue'>**Plot Results**</font>

fig_PnL = plt.figure(dpi= 125, facecolor='w')
fig_PnL.suptitle("Delta Hedging PnL \n", \
      fontweight="bold")
ax = fig_PnL.add_subplot()
ax.set_xlabel("PnL")
ax.set_ylabel("Frequency")
ax.hist(PNL_delta_multi, bins=30, \
          label=["Delta Hedging PnL"])
ax.legend()
plt.show()

In [ ]:
df_describe = pd.DataFrame(PNL_delta_multi)
df_describe.describe()

In [ ]:
#@title <font color='Blue'>**Run Quadratic Hedging!**</font>
PNL_quad_simple = np.zeros(num_paths)

for i in trange(num_paths):
    S_left_i = S_test_left[i,:].reshape(1,time_step+1)
    S_i = S_test[i,:].reshape(1,time_step+1)
    delta = Merton.quadratic_simple(S_left_i)
    cost = 0.0

    for j in range(time_step):
        PNL_quad_simple[i] += delta[j]*(S_i[0,j+1]-S_i[0,j])
        if j == 0:
          cost += np.abs(delta[j])*epsilon[0][0]*S_i[0,j]
        else: 
          cost += np.abs(delta[j]-delta[j-1])*epsilon[0][0]*S_i[0,j]

    if optiontype == "European":
        payoff = -payoff_func(S_i[0,-1])
    elif optiontype == "Barrier":
        payoff = -payoff_barrier(S_i,K,B,putorcall)
    PNL_quad_simple[i] = PNL_quad_simple[i] - payoff + price_C0 - cost

clear_output()

print("Finished computing the Quadratic-Hedging-Strategy!")

In [ ]:
df_describe = pd.DataFrame(PNL_quad_simple)
df_describe.describe()

In [ ]:
#@title <font color='Blue'>**Comparison Deep vs. Delta vs. Quadratic Hedging**</font>
df = pd.DataFrame(data={'Delta': PNL_delta_multi,  
                        'Quadratic-Underlying': PNL_quad_simple,
                        'Deep-Underlying': PNL_model1_simple,})
df_plot = pd.melt(df, value_vars=['Delta', 'Quadratic-Underlying', 'Deep-Underlying'], 
                  var_name='Hedge')
sns.displot(data=df_plot, hue='Hedge',x='value',
            kind="kde", fill=False, legend=True, height=5, aspect=1.6)
plt.title("Traditional vs Deep Hedging",fontweight="bold")
plt.xlim((x_limits))
plt.show()

In [ ]:
#@title <font color='Blue'>**Log Plot**</font>
diff = p[0] - price_C0
fig_PnL = plt.figure(figsize=(9,5), facecolor='w')
fig_PnL.suptitle("Traditional vs Deep Hedging - Log Scale",fontweight="bold")
ax = fig_PnL.add_subplot()
ax.set_xlabel("PnL")
ax.set_ylabel("Frequency")
ax.hist((PNL_delta_multi,PNL_quad_simple,PNL_model1_simple), bins=15, \
        color = ["0.7","0.4",(0,150/255,211/255)], \
        label=["Delta", "Quadratic with Underlying only", "Deep with Underlying only"],log=True)
ax.legend()
plt.show()

##Delta-Gamma Hedge and Quadratic Hedge

In [ ]:
#@title <font color='Blue'>**Compute Option Prices via Finite Differences.**</font>

opt_param_2 = Option_param(S0=S0, K=K_hedge, T=maturity, exercise="European", 
                           payoff=putorcall)
Merton_param_2 = Merton_process(r=risk_free, sig=sig, lam=lam, muJ=muJ, 
                                sigJ=sigJ)
Merton_2 = Merton_pricer(opt_param_2, Merton_param_2,optiontype="European",B=B,forhedge=False)
p2 = Merton_2.PIDE_price((Nspace,Ntime), Time=False) # Price at t=0
C_hedge_FD = p2[1] # Price matrix for all time steps and all prices of S
x_hedge = p2[2] 
# discretization of stock prices depends on strike, therefore x != x_hedge

print("Finished computing the Option Prices via FD!")

In [ ]:
#@title <font color='Blue'>**Run Delta-Gamma Hedging!**</font>
PNL_deltagamma_multi = np.zeros(num_paths)

for i in trange(num_paths):
    S_left_i = S_test_left[i,:].reshape(1,time_step+1)
    S_i = S_test[i,:].reshape(1,time_step+1)
    delta, gamma = Merton.deltagamma_hedge(S_left_i,x_hedge,C_hedge_FD)
    positions = Merton.get_pos(x_hedge,S_i)
    dt_hedge = dt
    cost = 0.0

    for j in range(time_step):
        pos = positions[j]
        pos_next = positions[j+1]
        get_timecol = int(np.round(j*dt_hedge/dt_FD))
        get_timecol_next = int(np.round((j+1)*dt_hedge/dt_FD))
        VH_t = C_hedge_FD[:,get_timecol]
        VH_tnext = C_hedge_FD[:,get_timecol_next]
        PNL_deltagamma_multi[i] += delta[j]*(S_i[0,j+1]-S_i[0,j]) + gamma[j]*(VH_tnext[pos_next]-VH_t[pos])
        if j == 0:
          cost += np.abs(gamma[j])*epsilon[0][1]*VH_t[pos]
        else: 
          cost += np.abs(gamma[j]-gamma[j-1])*epsilon[0][1]*VH_t[pos]

    if optiontype == "European":
        payoff = -payoff_func(S_i[0,-1])
    elif optiontype == "Barrier":
        payoff = -payoff_barrier(S_i,K,B,putorcall)
    
    PNL_deltagamma_multi[i] = PNL_deltagamma_multi[i] - payoff + p[0] - cost

clear_output()

print("Finished computing the Delta-Gamma-Hedging-Strategy!")

In [ ]:
#@title <font color='Blue'>**Plot Results**</font>

fig_PnL = plt.figure(dpi= 125, facecolor='w')
fig_PnL.suptitle("Delta-Gamma-Hedging PnL \n", \
      fontweight="bold")
ax = fig_PnL.add_subplot()
ax.set_xlabel("PnL")
ax.set_ylabel("Frequency")
ax.hist(PNL_deltagamma_multi, bins=30, \
          label=["Delta-Gamma-Hedging PnL"])
ax.legend()
plt.show()

In [ ]:
df_describe = pd.DataFrame(PNL_deltagamma_multi)
df_describe.describe()

In [ ]:
#@title <font color='Blue'>**Run Quadratic Hedging!**</font>
PNL_quad_multi = np.zeros(num_paths)

for i in trange(num_paths):
    S_left_i = S_test_left[i,:].reshape(1,time_step+1)
    S_i = S_test[i,:].reshape(1,time_step+1)
    quad = Merton.quadratic_multi(S_left_i,x_hedge,C_hedge_FD)
    positions = Merton.get_pos(x_hedge,S_i)
    dt_hedge = dt
    cost = 0.0

    for j in range(time_step):
        pos = positions[j]
        pos_next = positions[j+1]
        get_timecol = int(np.round(j*dt_hedge/dt_FD))
        get_timecol_next = int(np.round((j+1)*dt_hedge/dt_FD))
        VH_t = C_hedge_FD[:,get_timecol]
        VH_tnext = C_hedge_FD[:,get_timecol_next]
        PNL_quad_multi[i] += quad[j,0]*(S_i[0,j+1]-S_i[0,j]) + quad[j,1]*(VH_tnext[pos_next]-VH_t[pos])
        if j == 0:
          cost += np.abs(quad[j,1])*epsilon[0][1]*VH_t[pos]
        else: 
          cost += np.abs(quad[j,1]-quad[j-1,1])*epsilon[0][1]*VH_t[pos]

    if optiontype == "European":
        payoff = -payoff_func(S_i[0,-1])
    elif optiontype == "Barrier":
        payoff = -payoff_barrier(S_i,K,B,putorcall)
    PNL_quad_multi[i] = PNL_quad_multi[i] - payoff + p[0] - cost

clear_output()

print("Finished computing the Quadratic-Hedging-Strategy!")

In [ ]:
#@title <font color='Blue'>**Plot Results**</font>

fig_PnL = plt.figure(dpi= 125, facecolor='w')
fig_PnL.suptitle("Quadratic-Hedging PnL \n", \
      fontweight="bold")
ax = fig_PnL.add_subplot()
ax.set_xlabel("PnL")
ax.set_ylabel("Frequency")
ax.hist(PNL_quad_multi, bins=30, \
          label=["Quadratic-Hedging PnL"])
ax.legend()
plt.show()

In [ ]:
df_describe = pd.DataFrame(PNL_quad_multi)
df_describe.describe()

In [ ]:
#@title <font color='Blue'>**Comparison Deep vs. Delta-Gamma vs. Quadratic Hedging**</font>
diff = p[0] - price_C0
if run_recurrent == True:
    df = pd.DataFrame(data={'Delta-Gamma': PNL_deltagamma_multi, 
                            'Quadratic': PNL_quad_multi,
                            'Deep-Option simple': PNL_model2_simple, 
                            'Deep-Option recurrent': PNL_model2_recurrent})
    df_plot = pd.melt(df, value_vars=['Delta-Gamma','Quadratic','Deep-Option simple','Deep-Option recurrent'], 
                      var_name='Hedge')
    sns.displot(data=df_plot, hue='Hedge',x='value',
                kind="kde", fill=False, legend=True, height=5, aspect=1.6)
    plt.title("Traditional vs Deep Hedging",fontweight="bold")
    plt.xlim((x_limits))
    plt.show()
elif run_recurrent == False:
    df = pd.DataFrame(data={'Delta-Gamma': PNL_deltagamma_multi, 
                            'Quadratic-Option': PNL_quad_multi,
                            'Deep-Option': PNL_model2_simple})
    df_plot = pd.melt(df, value_vars=['Delta-Gamma', 'Quadratic-Option', 'Deep-Option'], 
                      var_name='Hedge')
    sns.displot(data=df_plot, hue='Hedge',x='value',
                kind="kde", fill=False, legend=True, height=5, aspect=1.6)
    plt.title("Traditional vs Deep Hedging",fontweight="bold")
    plt.xlim((x_limits))
    plt.show()

In [ ]:
#@title <font color='Blue'>**Log Plot**</font>
run_recurrent = False
if run_recurrent == True:
    fig_PnL = plt.figure(figsize=(9,5), facecolor='w')
    fig_PnL.suptitle("Traditional vs Deep Hedging - Log Scale",fontweight="bold")
    ax = fig_PnL.add_subplot()
    ax.set_xlabel("PnL")
    ax.set_ylabel("Frequency")
    ax.hist((PNL_deltagamma_multi,PNL_quad_multi,PNL_model2_simple,PNL_model2_recurrent), bins=15, \
            color = ["0.7","0.4",(0,150/255,211/255),(0,150/255,150/255)], \
            label=["Delta-Gamma", "Quadratic with Option", "Deep with Option (simple)","Deep with Option (recurrent)"],log=True)
    ax.legend()
    plt.show()
elif run_recurrent == False:
    fig_PnL = plt.figure(figsize=(9,5), facecolor='w')
    fig_PnL.suptitle("Traditional vs Deep Hedging - Log Scale",fontweight="bold")
    ax = fig_PnL.add_subplot()
    ax.set_xlabel("PnL")
    ax.set_ylabel("Frequency")
    ax.hist((PNL_deltagamma_multi,PNL_quad_multi,PNL_model2_simple), bins=15, \
            color = ["0.7","0.4",(0,150/255,211/255)], \
            label=["Delta-Gamma", "Quadratic with Option", "Deep with Option"],log=True)
    ax.legend()
    plt.xlim()
    plt.show()

In [ ]:
diff = p[0] - price_C0
if run_recurrent == True:
    df = pd.DataFrame(data={'Delta-Gamma': PNL_deltagamma_multi, 
                            'Deep-Option simple': PNL_model2_simple + diff, 
                            'Deep-Option recurrent': PNL_model2_recurrent + diff,
                            'Quadratic-Option': PNL_quad_multi})
    df_descrip = pd.melt(df, value_vars=['Delta-Gamma', 
                                         'Deep-Option simple',
                                         'Deep-Option recurrent','Quadratic-Option'], 
                         var_name='Hedge')
else:
    df = pd.DataFrame(data={'Delta-Gamma': PNL_deltagamma_multi, 
                            'Deep-Option': PNL_model2_simple + diff, 
                            'Quadratic-Option': PNL_quad_multi})
    df_descrip = pd.melt(df, value_vars=['Delta-Gamma', 
                                         'Deep-Option', 'Quadratic-Option'], 
                         var_name='Hedge')

df_descrip.groupby('Hedge').describe()

## Comparison of all

In [ ]:
#@title <font color='Blue'>**Plot All**</font>
diff = p[0] - price_C0
df = pd.DataFrame(data={'Delta': PNL_delta_multi, 
                       'Quadratic-Underlying': PNL_quad_simple,
                       'Deep-Underlying': PNL_model1_simple + diff, 
                       'Delta-Gamma': PNL_deltagamma_multi, 
                       'Quadratic-Option': PNL_quad_multi,
                       'Deep-Option': PNL_model2_simple + diff})
df_plot = pd.melt(df, value_vars=['Delta',
                                  'Quadratic-Underlying',
                                  'Deep-Underlying',
                                  'Delta-Gamma', 
                                  'Quadratic-Option',
                                  'Deep-Option'], 
                 var_name='Hedge')
sns.displot(data=df_plot, hue='Hedge',x='value',
            kind="kde", fill=False, legend=True, height=5, aspect=1.6)
plt.title("Traditional vs Deep Hedging",fontweight="bold")
plt.xlim((x_limits))
plt.show()

In [ ]:
#@title <font color='Blue'>**Descriptive Statistics**</font>
if run_recurrent == True:
    df = pd.DataFrame(data={'Delta-Underlying': PNL_delta_multi, 
                            'Deep-Underlying': PNL_model1_simple + diff, 
                            'Quadratic-Underlying': PNL_quad_simple,
                            'Delta-Gamma': PNL_deltagamma_multi, 
                            'Deep-Option simple': PNL_model2_simple + diff, 
                            'Deep-Option recurrent': PNL_model2_recurrent + diff,
                            'Quadratic-Option': PNL_quad_multi})
    df_descrip = pd.melt(df, value_vars=['Delta-Underlying','Deep-Underlying',
                                         'Quadratic-Underlying','Delta-Gamma', 
                                         'Deep-Option simple',
                                         'Deep-Option recurrent','Quadratic-Option'], 
                         var_name='Hedge')
else:
    df = pd.DataFrame(data={'Delta-Underlying': PNL_delta_multi, 
                            'Deep-Underlying': PNL_model1_simple + diff, 
                            'Quadratic-Underlying': PNL_quad_simple,
                            'Delta-Gamma': PNL_deltagamma_multi, 
                            'Deep-Option': PNL_model2_simple + diff, 
                            'Quadratic-Option': PNL_quad_multi})
    df_descrip = pd.melt(df, value_vars=['Delta-Underlying','Deep-Underlying',
                                         'Quadratic-Underlying','Delta-Gamma', 
                                         'Deep-Option', 'Quadratic-Option'], 
                         var_name='Hedge')

df_descrip.groupby('Hedge').describe()

In [ ]:
#@title <font color='Blue'>**Expected Shortfall**</font>
def ES(x,alpha):
    """
    Expected Shortfall
    x = PnL 
    alpha = Quantile
    """ 
    VaR = np.percentile(x, 100*(1 - alpha))
    CVaR = -x[x <= VaR].mean()
    return VaR, CVaR

In [ ]:
if run_recurrent: 
    results_VaR_ES = np.zeros((7,2))
    results_VaR_ES[0,:] = ES(PNL_delta_multi,0.95)
    results_VaR_ES[1,:] = ES(PNL_quad_simple,0.95)
    results_VaR_ES[2,:] = ES(PNL_model1_simple,0.95)
    results_VaR_ES[3,:] = ES(PNL_deltagamma_multi,0.95)
    results_VaR_ES[4,:] = ES(PNL_quad_multi,0.95)
    results_VaR_ES[5,:] = ES(PNL_model2_simple,0.95)
    results_VaR_ES[6,:] = ES(PNL_model2_recurrent,0.95)
else: 
    results_VaR_ES = np.zeros((6,2))
    results_VaR_ES[0,:] = ES(PNL_delta_multi,0.95)
    results_VaR_ES[1,:] = ES(PNL_quad_simple,0.95)
    results_VaR_ES[2,:] = ES(PNL_model1_simple,0.95)
    results_VaR_ES[3,:] = ES(PNL_deltagamma_multi,0.95)
    results_VaR_ES[4,:] = ES(PNL_quad_multi,0.95)
    results_VaR_ES[5,:] = ES(PNL_model2_simple,0.95)

In [ ]:
if run_recurrent:
    results_VaR_ES_2 = np.zeros((7,2))
    results_VaR_ES_2[0,:] = ES(-PNL_delta_multi,0.95)
    results_VaR_ES_2[1,:] = ES(-PNL_quad_simple,0.95)
    results_VaR_ES_2[2,:] = ES(-PNL_model1_simple,0.95)
    results_VaR_ES_2[3,:] = ES(-PNL_deltagamma_multi,0.95)
    results_VaR_ES_2[4,:] = ES(-PNL_quad_multi,0.95)
    results_VaR_ES_2[5,:] = ES(-PNL_model2_simple,0.95)
    results_VaR_ES_2[6,:] = ES(-PNL_model2_recurrent,0.95)
else: 
    results_VaR_ES_2 = np.zeros((6,2))
    results_VaR_ES_2[0,:] = ES(-PNL_delta_multi,0.95)
    results_VaR_ES_2[1,:] = ES(-PNL_quad_simple,0.95)
    results_VaR_ES_2[2,:] = ES(-PNL_model1_simple,0.95)
    results_VaR_ES_2[3,:] = ES(-PNL_deltagamma_multi,0.95)
    results_VaR_ES_2[4,:] = ES(-PNL_quad_multi,0.95)
    results_VaR_ES_2[5,:] = ES(-PNL_model2_simple,0.95)

In [ ]:
from tabulate import tabulate
if run_recurrent:
    df = [ ["Delta", results_VaR_ES[0,1], results_VaR_ES_2[0,1]],
        ["Quadratic Underlying", results_VaR_ES[1,1], results_VaR_ES_2[1,1]],
        ["Deep Underlying", results_VaR_ES[2,1], results_VaR_ES_2[2,1]],
        ["Delta-Gamma", results_VaR_ES[3,1], results_VaR_ES_2[3,1]],
        ["Quadratic Option", results_VaR_ES[4,1], results_VaR_ES_2[4,1]],
        ["Deep Option simple", results_VaR_ES[5,1], results_VaR_ES_2[5,1]],
        ["Deep Option recurrent", results_VaR_ES[6,1], results_VaR_ES_2[6,1]]]
else:
    df = [ ["Delta", results_VaR_ES[0,1], results_VaR_ES_2[0,1]],
        ["Quadratic Underlying", results_VaR_ES[1,1], results_VaR_ES_2[1,1]],
        ["Deep Underlying", results_VaR_ES[2,1], results_VaR_ES_2[2,1]],
        ["Delta-Gamma", results_VaR_ES[3,1], results_VaR_ES_2[3,1]],
        ["Quadratic Option", results_VaR_ES[4,1], results_VaR_ES_2[4,1]],
        ["Deep Option", results_VaR_ES[5,1], results_VaR_ES_2[5,1]]]

print(tabulate(df, headers=["Hedge", "negative 5% ES", "positive 5% ES"]))